# To solve a problem, define it

As programmers we are used to solving problems by coding up solutions. We do the hard work of figuring out a strategy and coding it up.

<img src="./application.jpg" style="display:inline"/>

Some problems can be solved by describing them. The description gets input to a general purpose solver. The solver does its thing and provides the answer we're after.

<img src="./solver.jpg" style="display:inline"/>

## Introducing Z3

The [Z3 solver](https://github.com/Z3Prover/z3/wiki) is a powerful [SMT solver](https://en.wikipedia.org/wiki/Satisfiability_modulo_theories). Source code is [available](https://github.com/Z3Prover/z3), as well as APIs for various languages -- including Python.

In [1]:
# First, install using pip.
!pip install z3-solver

In [2]:
# For convenience import Z3 into the global namespace
from z3 import *

## Example: Uniform Acceleration
We can provide the formulae for uniform acceleration as rules for a solver instance.

These rules describe the relationship between:
- `u`, initial speed
- `v`, final speed
- `a`, acceleration
- `t`, time
- `s`, distance

If we know any three of these, we can deduce the remaining pair.

<img src="./uniform-accel.jpg" style="display:inline"/>

In [3]:
u, v, s, a, t = Reals('u v s a t')

rules = [
    v == u + a * t,
    s == u * t + a * t * t / 2
]

solve(u == 10, a == 1, v == 20, *rules)

[s = 150, t = 10, v = 20, a = 1, u = 10]


## Example: Cryptarithms

<img src="./DudeneySendMoreMoney.png" style="display: inline" width=300px/>

The famous `SEND + MORE = MONEY` _cryptarithm_ by the British puzzler [Henry Ernest Dudeney](https://en.wikipedia.org/wiki/Henry_Dudeney) was published in The Strand magazine in 1924. 

An impoverished artist writes to their generous benefactor, asking for money. For discretion the artist uses letters rather than numbers. Each letter represents a single digit.

Exactly how much money is the artist asking for?
Is there more than one solution? 

In [4]:
# Create the required digits 
S, E, N, D, M, O, R, Y = Ints('S E N D M O R Y')

def in_range(lo, hi, *vars):
    return [And(lo <= var, var < hi) for var in vars]

s = Solver()
# Add a constraint
s.add(        1000*S + 100*E + 10*N + D + 
              1000*M + 100*O + 10*R + E == 
    10000*M + 1000*O + 100*N + 10*E + Y)
s.add(Distinct(S, E, N, D, M, O, R, Y))
s.add(in_range(0, 10, S, E, N, D, M, O, R, Y))
s.add(in_range(1, 10, S, M))

def amount(model, *digits):
    v = 0
    for digit in digits:
        v = v * 10 + model[digit].as_long()
    return v

s.check()
amount(s.model(), M, O, N, E, Y)

10652

## Example: Sudoku

This is a classic way to demonstrate the power of solvers. The constraints are easy to express. There are two types of constraint:
- the generic rules (each row/column/block is a permutation of 1, 2, ... 9)
- the initial setup for this instance

<img src="./sudoku.png" style="display: inline"/>

In [5]:
from itertools import *
sq3x3 = list(product(range(3), repeat=2))
sq3x3

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [6]:
# The 9x9 board has: 
# - 81 cells
# - 9 rows of 9 cells
# - 9 columns of 9 cells
# - and 3x3 blocks of 3x3 cells

puzzle = '''\
   | 94| 3 
   |51 |  7
 89|   | 4 
---+---+---
   |   |2 8
 6 |2 1| 5 
1 2|   |   
---+---+---
 7 |   |52 
9  | 65|   
 4 |97 |   
'''

cells = [Int(str(s)) for s in range(81)]

def display(model):
    'Return a string representation of the Sudoku model'
    vs = count()
    return ''.join(
        str(model[cells[next(vs)]]) if c in ' 123456789' else c
        for c in puzzle)

def cell(R, C, r, c):
    'Return the cell at position (r,c) in block (R,C)'
    return cells[9*(3*R+r) + 3*C + c]

# Rows, columns, blocks have no repeats
rows = [Distinct(cells[r:r+9]) for r in range(0, 81, 9)]
cols = [Distinct(cells[c::9]) for c in range(9)]
blocks = [Distinct([cell(*p, *d) for d in sq3x3]) for p in sq3x3]

sudoku_rules = rows + cols + blocks

s = Solver()
s.add(sudoku_rules)
s.add(in_range(1, 10, *cells))

start_cells = (c for c in puzzle if c in ' 123456789')
for i, c in enumerate(start_cells):
    if c in '123456789':
        s.add(cells[i]==int(c))
s.add(cells[0] != 5)
s.check()

print(display(s.model()))

715|894|632
234|516|897
689|723|145
---+---+---
493|657|218
867|231|954
152|489|763
---+---+---
376|148|529
928|365|471
541|972|386



## Example: Knapsack Problem

This is a classic optimisation problem, more specifically a packing problem. 

We have some items to take in our knapsack. Each item has a `value` and a `weight`. The knapsack has a `capacity`, the maximum weight it can carry. Which items should we select to maximise value?

In [7]:
values = [360, 83, 59, 130, 431, 67, 230, 52]
weights = [7, 0, 30, 22, 80, 94, 11, 81]
capacity = 80

# selections[n] True => we put the nth item in the knapsack
selections = [Bool(f'item{n}') for n in range(len(values))]

knapsack = Optimize()

# Total weight does not exceed capacity
knapsack.add(sum(s * w for s, w in zip(selections, weights)) <= capacity)

# Maximise the total value
objective = knapsack.maximize(sum(s * v for s, v in zip(selections, values)))

knapsack.check(), knapsack.model(), objective.value()

(sat,
 [item5 = False,
  item7 = False,
  item4 = False,
  item0 = True,
  item6 = True,
  item2 = True,
  item3 = True,
  item1 = True],
 862)

## Notes

The knapsack problem is an important optimisation problem.

It's not just about packing things. Many similar problems have the same solution.

Examples:
- A family is dining out on a budget. Everyone has their favourite starter, main, desert. What will they order?
- The product manager has a delivery date and a list of features. Each feature has a cost (story points) and a value. We know the team velocity. What's the plan?

![Menu choice](menu.jpg)

For real life problems, there will be extra constraints. E.g. everyone gets a main course, or some choices are sold out. For the product, maybe we have to include the "user registration" feature, or features might be inter-dependent ...

These constraints can be added without needing to modify the solver. In fact, they help the solver get an answer! Typically with real world problems the variables are not truly independent. Because of this a solution which is expensive to calculate, in theory, can be found in practice.

Other problems well suited to solvers include dependency management, scheduling, routing ...

Note also that a generic solver may not be the best route to an answer. In the case of the knapsack problem there are more specialised (and efficient) algorithms. It is possible to configure and extend generic solvers too, e.g. to favour particular strategies.

If you're interested in how SMT solvers work, SAT solvers are a good starting point. SAT solvers determine if a set of expressions using boolean variables can be _satisfied_. [MiniSAT](http://minisat.se/) is a minimalistic production quality SAT solver developed for expositional purposes.